# 12-订单


## 1. 订单分析
* 1. 订单表  order_id 是主键 查询 用户靠order_id  让后台查询 
* 2. 订单商品表  sku多个不确定 外键属性

* 1. 复制模型类 orders.models.py -
* 2. 数据迁移


## 2.保存订单
#### 订单的分析流程
*    1. 接收解析解析参数 address_id payt_method
*    2. 校验 1.判空 2.地址是否有 3. pay-method是否 
*    3. 生成唯一 时间订单号
*    4. 创建orderInfo这个数据 order,user,address,pay_method, freight: Deciaml(10.0)

*    5. 获取购物车redis数据 ; 商品必须选中状态
*    6. 根据购物车 sku_ids 获取 skus商品 SKU
*    7. 判断sku的库存量  购买的个数count > 库存量sku.stock: 购买失败
*    8. sku 库存量减少, 销量增加
*    9. spu 销量增加
*   10. 计算该sku的总价格, 总个数;
*  11. 订单商品表 创建 order,sku有关系属性
*  12. 累加运费, orderInfo.save()
*  13. 清空选中购物车数据 redis_client.hdel(user.id,*carts_selected_dict)
*  14. 返回前端的响应结果


## 3.提交订单 成功:
*   1.urls.py
*   2. views.py 
*   3. 解析参数 传递前端 request.GET
*   4. return render(request,'order_sucess.html',context)


## 4.事物处理
* 1次完整的订单提交 需要经历4张表 ; 任意的一次意外生成的数据都是不完整的 不记录; 事务处理
* mysql 
    * 1.开启事务 begin
    * 2.提交事务 commit
    * 3.事务回滚 rollback

* django-orm :数据库 交互 
    * 1.开启事务 with transaction.atomic():
    * 2.提交事务 savepoint_commit
    * 3.事务回滚 savepoint_rollback
    * 4. 设置保存点: save_id = savepoint()
    * 暴力回滚--只要出现异常 全部回滚

    
## 5.乐观锁
    * 1.加锁----资源竞争
    * 2.悲观锁--互斥锁--容易死锁
    * 3.乐观锁-- SKU.objects.filter(id=1,stock=老库存).update(stock=2)

1. 多用户资源争夺 数据库的 同一份
2. 让前面的用户操作的时候, 后面的用户进行等待
3. 悲观锁--互斥锁-->容易死锁
4. 乐观锁-- 必须满足两个条件 , user.id; 当前库存是不是原始库存; 如果是 才继续操作
5. 队列--并行->串行-->celery-redis(rabittMQ)--秒杀 竞拍

1. 实现乐观锁
1. SKU.objects.filter(id=sku.id,stock=原始库存).update(stock=new_stock,sales=new_sales)
2. 获取原始的 库存 和销量
3. 在原始库存 和销量 基础上 计算
4. if result == 0: 下单失败: 1.库存不足 2,库存足 但是 发生改变了 continue:
5. 如果下单成功 break

## 6.设置 mysql的事务隔离级别
4种
1. 串行排队
2. 可重复读: 第1个事务修改了数据库数据, 第2个事务不知晓
3. 已经提交: 第1个事务修改了数据库数据, 第2个事务知晓
4. 未提交: 第1个事务修改了数据,但是未提交库数据, 第2个事务知晓了修改后的结果 

操作:设置mysql事务隔离级别为:已经提交

transaction 事务-->一连串的数据库操作 必须同时 满足插入条件 才可以; 如果有任何一个表失败 就事务回滚

SQL: 开启事务 begin
     提交事务 commit 
     事务回滚 rollback

     暴力回滚
django:orm 新知识点
    from django.db import transaction

   
    开启事务  with transaction.atomic():
             save_id = transaction.save_point()

    提交事务 transaction.savepoint_commit (save_id)
    事务回滚 transaction.savepoint_rollback(save_id)

    悲观锁(互斥锁)---乐观锁----队列


    1. 确认 有两个用户
    2. 两个不同的浏览器 火狐 IE 360 safari chrome
    3 .改 商品 库存 
    4. 在 views.py -- 判断完毕库存--time.sleep(10)
    4. 分别在不同浏览器--不同用户---操作 同一个商品 进行提交订单

    1000--3 --3
    for sku in skus:

    result = SKU.objects.fitle(id=sku_id,stock="以前的库存").update(stock=,sales=)
    if result == 0:
        continue



    # 1.注册支付宝的开发者
    # 2.注册应用---沙箱测试环境

    # 3. 美多商城 生成秘钥---将美多公钥--支付宝
                        ---支付宝公钥-->美多

    # 4. 1. 获取支付宝 支付网址----支付--买家---->
    # 5. 2. 买完毕---美多URL--->绑定交易ID( 支付表)


    #1.在本地终端  生成 秘钥 美多商城的---app_private_key.pem   app_public_key.pem 
    # 2. 创建 payment子应用 -- 秘钥拖进来
    # 3. 剪切: app_public_key.pem 内容------->支付宝
    # 4. 赋值支付宝的公钥------美多商城里面


    # 1.获取支付宝的 支付网址
    # 1.装包 pip install python-alipay-sdk QQLoginTool 
    # 2. order_id --校验 
    # 3. alipay对象
    # 4. 生成支付宝网址: '支付宝网关' + order_string
    # 5. 返回响应




